In [4]:
import pandas as pd
import scipy.spatial.distance as sci_dist
import matplotlib
matplotlib.use('QtAgg')
import seaborn as sns
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib
Image.MAX_IMAGE_PIXELS = None
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson
import os
import re
np.random.seed(0)
matplotlib.rcParams['pdf.fonttype'] = 42

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']

data_folder = '/Users/yaelheyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_2/exports'
region_data = '/Users/YaelHeyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/Region_Data_with_Minimal_Values.csv'

cell_by_gene_path = data_folder + '/cell_by_gene_matrix_dilate10_20240718_withbarcodes_atleast3.csv'
transcripts_path = data_folder + '/decode_1_withcoordinates.csv'
coords_path = data_folder + '/segmentation_1_nuclei_cellattributes.csv'
output_folder = data_folder + '/processedData'
cell_by_gene = pd.read_csv(cell_by_gene_path)


In [5]:
clones = np.empty(0)
num_cells = np.empty(0)
pixel2um = 107.11 / 1000000  # Convert pixel to micrometers
combined_df = pd.DataFrame()
region_data = pd.read_csv(region_data)
for j in range(1,4):  # Adjust range as needed
    # Dynamically construct the path
    data_folder = f"/Users/yaelheyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_{j}/exports"
    cell_by_gene_path = os.path.join(data_folder, 'cell_by_gene_matrix_dilate10_20240718_withbarcodes_atleast3.csv')
    print(data_folder)
    # Load the cell by gene data
    cell_by_gene = pd.read_csv(cell_by_gene_path, index_col=0)
    
    # Get reference X and Y from region_data as scalar values
    ref_x = region_data.loc[region_data['Region'] == f"Region {j}", "Min X (mm)"].iloc[0]
    ref_y = region_data.loc[region_data['Region'] == f"Region {j}", "Min Y (mm)"].iloc[0]
    # clones = np.append(clones, len(np.unique(cell_by_gene.loc[cell_by_gene['bc_cluster_n_bcs']>2,'bc_cluster'])))
    num_cells = np.append(num_cells, len(cell_by_gene))
    # Correct the coordinates
    cell_by_gene['center_x'] = (cell_by_gene['center_x'] * pixel2um + ref_x)*1000
    cell_by_gene['center_y'] = (cell_by_gene['center_y'] * pixel2um + ref_y)*1000
    
    # Adjust the cell_id to make them unique across regions
    cell_by_gene['cell_id'] = cell_by_gene['cell_id'] + j * 100000
    
    # Concatenate to the combined DataFrame
    combined_df = pd.concat([combined_df, cell_by_gene], ignore_index=True)


/Users/yaelheyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_1/exports
/Users/yaelheyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_2/exports
/Users/yaelheyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-05-21_mouseexp_expression/projects/roi_3/exports


In [6]:
# --- Barcode columns: bc_ followed by exactly 3 digits ---
barcode_cols = [col for col in combined_df.columns if re.fullmatch(r'bc_\d{3}', col)]
barcode_data = combined_df[barcode_cols].apply(pd.to_numeric, errors='coerce')

# Step 1: Filter values < 3
barcode_filtered = barcode_data.mask(barcode_data < 3, np.nan)

# Step 2: Normalize filtered values by area
barcode_filtered_norm = barcode_filtered.div(combined_df['area'], axis=0)

# Step 3: Compute means
barcode_means = barcode_filtered.mean(skipna=True)
barcode_means_norm = barcode_filtered_norm.mean(skipna=True)

# --- Gene columns: all uppercase letters and digits only ---
gene_cols = [col for col in combined_df.columns if re.fullmatch(r'[A-Z0-9]+', col)]
gene_data = combined_df[gene_cols].apply(pd.to_numeric, errors='coerce')

# Step 1: Filter values < 3
gene_filtered = gene_data.mask(gene_data < 3, np.nan)

# Step 2: Normalize filtered values by area
gene_filtered_norm = gene_filtered.div(combined_df['area'], axis=0)

# Step 3: Compute means
gene_means = gene_filtered.mean(skipna=True)
gene_means_norm = gene_filtered_norm.mean(skipna=True)

# --- Print summary ---
print("Barcode mean (raw ≥3):", np.mean(barcode_means))
print("Barcode mean (per area ≥3):", np.mean(barcode_means_norm))

print("Gene mean (raw ≥3):", np.mean(gene_means))
print("Gene mean (per area ≥3):", np.mean(gene_means_norm))

# --- Save to CSV ---
barcode_summary = pd.DataFrame({'mean': barcode_means, 'mean_per_area': barcode_means_norm})
barcode_summary.to_csv(os.path.join(output_folder, "barcode_summary.csv"))

gene_summary = pd.DataFrame({'mean': gene_means, 'mean_per_area': gene_means_norm})
gene_summary.to_csv(os.path.join(output_folder, "gene_summary.csv"))


Barcode mean (raw ≥3): 10.19273410605982
Barcode mean (per area ≥3): 0.0009384751548414902
Gene mean (raw ≥3): 5.188219979328732
Gene mean (per area ≥3): 0.0004441266938306024


In [7]:

gene_stds = gene_filtered.std(skipna=True)
gene_stds
top15_std = gene_stds.sort_values(ascending=False).head(10)
top15_std

SPARC       16.466009
MMP1        16.026539
FN1         15.155689
CCND1       14.944136
L1CAM       14.470032
COL1A1      14.425744
CAPG        13.003327
SERPINE1    12.537876
IGFBP7      11.827279
BGN         10.973078
dtype: float64